<a href="https://colab.research.google.com/github/Pororocode/Bangkit-C22CB-Company-Based-Capstone/blob/main/ML/bert_squad-id_transfer_learn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [112]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Thu Jun  2 03:40:40 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P0    35W / 250W |   3169MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/dhupee/Bangkit-C22CB-Company-Based-Capstone/blob/30b0995970f29114749cff04deef444de6832993/ML/distilbert_transfer_learn.ipynb)

In [113]:
# check python version
import sys
print(sys.version)

3.7.13 (default, Apr 24 2022, 01:04:09) 
[GCC 7.5.0]


In [ ]:
# notebook settings

COLAB_MODE = False # set to True if running in Google Colab
ENABLE_JSON2CSV = False # set to True if you want to convert json dataset to csv, DOESNT NEEDED ANYMORE

In [ ]:
# if COLAB_MODE is True, then work around the repository
if COLAB_MODE:
    import os
    branch_name = 'dhupee-dev'
    cloned_repo_name = 'remote-clone'
    target_repo_dir = '/content/remote-clone/ML'
    repo_link = 'https://github.com/dhupee/Bangkit-C22CB-Company-Based-Capstone.git'
    # if current directory is not the cloned repo, clone it
    if not os.path.exists(target_repo_dir):
        !git clone --single-branch --branch $branch_name $repo_link $cloned_repo_name
        print('Repo successfully cloned!')
        %cd $target_repo_dir
        %pwd
    else:
        print('Repo already cloned')

In [114]:
# check if transformers and tensorflow are installed, if not install them
# use transformers version 4.18.0 and tensorflow version 2.8.0
try:
    import transformers
    import tensorflow as tf
    print("transformers and tensorflow are installed")
except:
    print("transformers and tensorflow are not installed")
    print("installing transformers and tensorflow")
    # install transformers 4.18.0 and tensorflow 2.8.0
    %pip install transformers==4.18.0
    %pip install tensorflow==2.8.1
    # import transformers and tensorflow again
    import transformers
    import tensorflow as tf

transformers and tensorflow are installed


In [273]:
# loading base model

model_name = "cahya/bert-base-indonesian-522M"
batch_size = 16

from transformers import AutoTokenizer, TFAutoModel # make sure use tensorflow model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = TFAutoModel.from_pretrained(model_name) # if not specified, it will use torch model

Some layers from the model checkpoint at cahya/bert-base-indonesian-522M were not used when initializing TFBertModel: ['mlm___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at cahya/bert-base-indonesian-522M.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [274]:
model.summary()

Model: "tf_bert_model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
Total params: 110,617,344
Trainable params: 110,617,344
Non-trainable params: 0
_________________________________________________________________


In [275]:
# test tokenizer
tokenizer("Nama kamu siapa?")

{'input_ids': [3, 1769, 8343, 6186, 32, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1]}

In [276]:
tokenizer("saya suka makan nasi goreng")

{'input_ids': [3, 3245, 5366, 2464, 6014, 11186, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}

In [277]:
# see how base model works
unmasker = transformers.pipeline('fill-mask', model = model_name)
unmasker("mainan saya [MASK] di jalan")

[{'score': 0.08403640985488892,
  'sequence': 'mainan saya berada di jalan',
  'token': 2186,
  'token_str': 'berada'},
 {'score': 0.07038332521915436,
  'sequence': 'mainan saya ada di jalan',
  'token': 1821,
  'token_str': 'ada'},
 {'score': 0.04035765677690506,
  'sequence': 'mainan saya sendiri di jalan',
  'token': 1998,
  'token_str': 'sendiri'},
 {'score': 0.02904832921922207,
  'sequence': 'mainan saya lahir di jalan',
  'token': 2444,
  'token_str': 'lahir'},
 {'score': 0.028137262910604477,
  'sequence': 'mainan saya berdiri di jalan',
  'token': 3812,
  'token_str': 'berdiri'}]

In [ ]:
#this for converting the file to huggingface format
#I suggest you to do it seperately

def convert_huggingface(dataset_path):
    import json
    with open(dataset_path, encoding="utf8") as f:
        content = json.load(f)

    hf_data = []
    for data in content["data"]:
        title = data["title"]
        for paragraph in data["paragraphs"]:
            context = paragraph["context"]
            for qa in paragraph["qas"]:
                fill = {
                    "id": qa["id"],
                    "title": title,
                    "context": context,
                    "question": qa["question"],
                    "answers": {"answer_start": [], "text": []}
                }
                if qa["is_impossible"]:
                    answers = qa["plausible_answers"]
                else:
                    answers = qa["answers"]
                for answer in answers:
                    fill["answers"]["answer_start"].append(answer["answer_start"])
                    fill["answers"]["text"].append(answer["text"])

                hf_data.append(fill)
    # Add "_hf" before .json extension
    hf_dataset_path = dataset_path.replace(".json", "_hf.json")
    with open(hf_dataset_path, "w") as f:
        json.dump({"data": hf_data}, f)

    return hf_data

In [278]:
# load dataset json file
import json

#colab pro + drive
train_json_dir = "/content/drive/MyDrive/Translated/hf_train-v2.0_indo.json"
valid_json_dir = "/content/drive/MyDrive/Translated/hf_dev-v2.0_indo.json"
tester_json_dir  = "/content/drive/MyDrive/Translated/tester_indo.json"

dataset_dirs = [train_json_dir, valid_json_dir, tester_json_dir]
# dataset_dirs = [tester_json_dir]

In [279]:
# importing dataset

try:
    from datasets import load_dataset
except:
    print("datasets module not found")
    print("installing dataset module")
    %pip install datasets
    from datasets import load_dataset

datasets = load_dataset(
    'json',
    data_files={'train': train_json_dir, 'validation': valid_json_dir},
    field='data'
    )

Using custom data configuration default-2599678fccd9e0e7
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-2599678fccd9e0e7/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/2 [00:00<?, ?it/s]

In [280]:
# sample of dataset randomly
import random
datasets["train"][4]

{'answers': {'answer_start': [304], 'text': ['akhir 1990-an']},
 'context': 'Beyoncé Giselle Knowles-Carter (/ biːˈjɒnseɪ / bee-YON-say) (lahir 4 September 1981) adalah seorang penyanyi, penulis lagu, produser rekaman dan aktris Amerika. Dilahirkan dan dibesarkan di Houston, Texas, ia tampil di berbagai kompetisi menyanyi dan menari sebagai seorang anak, dan mulai terkenal pada akhir 1990-an sebagai penyanyi utama dari grup wanita R&B Destiny\'s Child. Dikelola oleh ayahnya, Mathew Knowles, grup ini menjadi salah satu grup wanita terlaris di dunia sepanjang masa. Jeda mereka melihat perilisan album debut Beyoncé, Dangerously in Love (2003), yang menjadikannya sebagai artis solo di seluruh dunia, meraih lima Grammy Awards dan menampilkan single nomor satu Billboard Hot 100 "Crazy in Love" dan "Baby Boy" .',
 'id': '56bf6b0f3aeaaa14008c9602',
 'question': 'Pada dekade berapa Beyonce menjadi terkenal?',
 'title': 'Beyoncé'}

In [281]:
# sample of dataset randomly
import random
datasets["validation"][4]

{'answers': {'answer_start': [671, 649, 671, 671],
  'text': ['abad ke-10',
   'paruh pertama abad ke-10',
   'tanggal 10',
   'tanggal 10']},
 'context': 'Bangsa Norman (Norman: Nourmands; Prancis: Normandia; Latin: Normanni) adalah orang-orang yang pada abad ke-10 dan ke-11 memberi nama kepada Normandia, sebuah wilayah di Prancis. Mereka adalah keturunan dari perampok dan bajak laut Norse ("Norman" berasal dari "Norseman") dan bajak laut dari Denmark, Islandia dan Norwegia yang, di bawah pemimpin mereka Rollo, setuju untuk bersumpah setia kepada Raja Charles III dari Francia Barat. Melalui generasi asimilasi dan pencampuran dengan penduduk asli Franka dan Romawi-Gaul, keturunan mereka secara bertahap akan bergabung dengan budaya berbasis Carolingian di Francia Barat. Identitas budaya dan etnis yang berbeda dari Normandia awalnya muncul pada paruh pertama abad ke-10, dan terus berkembang selama abad-abad berikutnya.',
 'id': '56ddde6b9a695914005b962c',
 'question': 'Abad berapa orang 

In [282]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML


def show_random_elements(dataset, num_examples=10):
    '''display random elements from dataset

    Args:
        dataset (Dataset): dataset to show
        num_examples (int, optional): number of examples to show. Defaults to 10.
    '''
    assert num_examples <= len(
        dataset
    ), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset) - 1)
        while pick in picks:
            pick = random.randint(0, len(dataset) - 1)
        picks.append(pick)

    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(
                lambda x: [typ.feature.names[i] for i in x]
            )
    display(HTML(df.to_html()))

In [283]:
show_random_elements(datasets["train"]) # brace yourself, this is gonna be a long list

,id,title,context,question,answers
0,5a45a2c619a820001a1edacc,Nonprofit_organization,"Pekerjaan sektor publik dan swasta, sebagian besar, mampu menawarkan lebih banyak untuk karyawan mereka daripada kebanyakan lembaga nonprofit sepanjang sejarah. Baik dalam bentuk upah yang lebih tinggi, paket manfaat yang lebih komprehensif, atau pekerjaan yang tidak terlalu membosankan, sektor publik dan swasta telah menikmati keuntungan dalam menarik karyawan daripada NPO. Secara tradisional, NPO telah menarik individu yang digerakkan oleh misi yang ingin membantu tujuan pilihan mereka. Yang memperparah masalah adalah bahwa beberapa NPO tidak beroperasi dengan cara yang mirip dengan kebanyakan bisnis, atau hanya secara musiman. Hal ini menyebabkan banyak karyawan muda dan terdorong untuk melepaskan NPO demi pekerjaan yang lebih stabil. Namun saat ini, organisasi Nonprofit mengadopsi metode yang digunakan oleh pesaing mereka dan menemukan cara baru untuk mempertahankan karyawan mereka dan menarik tenaga kerja terbaik dari yang baru dibentuk.",Jenis karyawan apa yang menarik sektor swasta?,"{'answer_start': [416], 'text': ['individu yang digerakkan oleh misi yang ingin membantu tujuan pilihan mereka']}"
1,5a7b534b21c2de001afe9ebc,Economy_of_Greece,"Dalam beberapa tahun terakhir sejumlah organisasi terkait pariwisata terkenal telah menempatkan tujuan Yunani di bagian atas daftar mereka. Pada tahun 2009 Lonely Planet memberi peringkat Thessaloniki, kota terbesar kedua di negara itu, ""Kota Pesta Terbaik"" terbaik kelima di dunia, di samping kota-kota seperti Montreal dan Dubai, sementara pada tahun 2011 pulau Santorini terpilih sebagai pulau terbaik di dunia oleh Travel + Kenyamanan. Pulau tetangga Mykonos menduduki peringkat ke-5 pulau terbaik Eropa. Thessaloniki adalah Ibukota Pemuda Eropa pada tahun 2014.",Berapa banyak organisasi terkait pariwisata yang menempatkan tujuan Yunani di urutan terbawah?,"{'answer_start': [125], 'text': ['daftar mereka']}"
2,5a69098b8476ee001a58a9fc,IBM,"IBM mengumumkan akan meluncurkan perangkat lunak barunya, yang disebut ""Penawaran Klien Terbuka"" yang akan berjalan di Linux, Microsoft Windows dan Apple Mac OS X. Perusahaan tersebut menyatakan bahwa produk barunya memungkinkan perusahaan untuk menawarkan karyawan pilihan untuk menggunakan perangkat lunak yang sama pada Windows dan alternatifnya. Ini berarti bahwa ""Penawaran Klien Terbuka"" adalah untuk memotong biaya pengelolaan apakah akan menggunakan Linux atau Apple dibandingkan dengan Windows. Tidak ada keharusan bagi perusahaan untuk membayar Microsoft atas lisensinya untuk sistem operasi karena sistem operasi tidak akan lagi bergantung pada perangkat lunak yang berbasis Windows. Salah satu alternatif untuk format dokumen kantor Microsoft adalah perangkat lunak Open Document Format, yang pengembangannya didukung oleh IBM. Ini akan digunakan untuk beberapa tugas seperti: pengolah kata, presentasi, bersama dengan kolaborasi dengan Lotus Notes, alat olahpesan cepat dan blog serta pesaing Internet Explorer - browser web Mozilla Firefox. IBM berencana untuk menginstal Open Client pada 5% PC desktopnya. Penawaran Linux telah tersedia sebagai produk Klien IBM untuk Pekerjaan Cerdas pada platform Ubuntu dan Red Hat Enterprise Linux.",Apa alternatif untuk menggunakan format dokumen Linux?,"{'answer_start': [762], 'text': ['perangkat lunak Open Document Format,']}"
3,5ad157aa645df0001a2d1800,Mary_(mother_of_Jesus),"Devosi pada lukisan artistik Maria berbeda-beda di antara tradisi Kristen. Ada tradisi panjang seni Maria Katolik Roma dan tidak ada gambar yang menembus seni Katolik seperti halnya gambar Madonna dan Child. Ikon Perawan Theotokos bersama Kristus tidak diragukan lagi adalah ikon yang paling dihormati di Gereja Ortodoks. Baik Katolik Roma maupun Kristen Ortodoks menghormati gambar dan ikon Maria, mengingat Konsili Nicea Kedua pada tahun 787 mengizinkan pemujaan mereka dengan pemahaman bahwa mereka yang menghormati gambar itu menghormati realitas orang y

In [ ]:
'''
    This function is for converting SQuAD json file to pandas dataframe, iteratively

    I dont want run this locally, better use colab

    doesn't needed anymore, use load_dataset instead
'''

if ENABLE_JSON2CSV:
    import utils
    for dir in dataset_dirs:
        with open(dir, encoding="utf-8") as json_file:
            file = json.load(json_file)
            dict_file = file
            data = dict_file['data']

        df = utils.json_to_df(data)
        df.to_csv(dir.replace(".json", ".csv"), index = False)

In [284]:
assert isinstance(tokenizer, transformers.PreTrainedTokenizerFast) # make sure tokenizer is pre-trained

---

In [285]:
max_length = 384  # The maximum length of a feature (question and context)
doc_stride = 128  # The allowed overlap between two part of the context when splitting is performed.

In [286]:
# check if there's dataset feature
# longer than max_length
for i, example in enumerate(datasets["train"]):
    if len(tokenizer(example["question"], example["context"])["input_ids"]) > 384:
        break
example = datasets["train"][i]

In [287]:
# check if there's dataset feature longer than max_length
len(tokenizer(example["question"], example["context"])["input_ids"])

402

In [288]:
# check truncate dataset length
len(
    tokenizer(
        example["question"],
        example["context"],
        max_length=max_length,
        truncation="only_second",
    )["input_ids"]
)

384

In [289]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    stride=doc_stride,
)

In [290]:
[len(x) for x in tokenized_example["input_ids"]]

[384, 156]

In [291]:
for x in tokenized_example["input_ids"][:2]:
    print(tokenizer.decode(x))

[CLS] beyonce menikah pada 2008 dengan siapa? [SEP] pada 4 april 2008, beyonce menikahi jay z. dia secara terbuka mengungkapkan pernikahan mereka dalam montase video di pesta mendengarkan untuk album studio ketiganya, i am... sasha fierce, di sony club manhattan pada 22 oktober 2008. i am... sasha fierce dirilis pada 18 november 2008 di amerika serikat. album ini secara resmi memperkenalkan alter ego beyonce sasha fierce, yang dibuat selama pembuatan singel tahun 2003 " crazy in love ", terjual 482. 000 kopi di minggu pertama, memulai debutnya di atas billboard 200, dan memberikan beyonce album nomor satu ketiganya berturut - turut di kami. album ini menampilkan lagu nomor satu " single ladies ( put a ring on it ) " dan lagu lima teratas " if i were a boy " dan " halo ". mencapai pencapaian menjadi single hot 100 terlama dalam karirnya, kesuksesan " halo " di as membantu beyonce mencapai lebih dari sepuluh single teratas dalam daftar daripada wanita lain selama tahun 2000 - an. ini jug

In [292]:
tokenized_example = tokenizer(
    example["question"],
    example["context"],
    max_length=max_length,
    truncation="only_second",
    return_overflowing_tokens=True,
    return_offsets_mapping=True,
    stride=doc_stride,
)
print(tokenized_example["offset_mapping"][0][:100])

[(0, 0), (0, 7), (8, 15), (16, 20), (21, 25), (26, 32), (33, 38), (38, 39), (0, 0), (0, 4), (5, 6), (7, 12), (13, 17), (17, 18), (19, 26), (27, 35), (36, 39), (40, 41), (41, 42), (43, 46), (47, 53), (54, 61), (62, 75), (76, 86), (87, 93), (94, 99), (100, 104), (104, 107), (108, 113), (114, 116), (117, 122), (123, 135), (136, 141), (142, 147), (148, 154), (155, 164), (164, 165), (166, 167), (168, 170), (171, 172), (172, 173), (173, 174), (175, 178), (178, 180), (181, 183), (183, 186), (186, 187), (187, 188), (189, 191), (192, 196), (197, 201), (202, 211), (212, 216), (217, 219), (220, 227), (228, 232), (232, 233), (234, 235), (236, 238), (239, 240), (240, 241), (241, 242), (243, 246), (246, 248), (249, 251), (251, 254), (254, 255), (256, 263), (264, 268), (269, 271), (272, 280), (281, 285), (286, 288), (289, 296), (297, 304), (304, 305), (306, 311), (312, 315), (316, 322), (323, 328), (329, 343), (344, 349), (350, 353), (354, 361), (362, 365), (365, 367), (368, 370), (370, 373), (373, 3

In [293]:
sequence_ids = tokenized_example.sequence_ids()
print(sequence_ids)

[None, 0, 0, 0, 0, 0, 0, 0, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 

In [294]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [295]:
answers = example["answers"]
start_char = answers["answer_start"][0]
end_char = start_char + len(answers["text"][0])

# Start token index of the current span in the text.
token_start_index = 0
while sequence_ids[token_start_index] != 1:
    token_start_index += 1

# End token index of the current span in the text.
token_end_index = len(tokenized_example["input_ids"][0]) - 1
while sequence_ids[token_end_index] != 1:
    token_end_index -= 1

# Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
offsets = tokenized_example["offset_mapping"][0]
if (
    offsets[token_start_index][0] <= start_char
    and offsets[token_end_index][1] >= end_char
):
    # Move the token_start_index and token_end_index to the two ends of the answer.
    # Note: we could go after the last offset if the answer is the last word (edge case).
    while (
        token_start_index < len(offsets) and offsets[token_start_index][0] <= start_char
    ):
        token_start_index += 1
    start_position = token_start_index - 1
    while offsets[token_end_index][1] >= end_char:
        token_end_index -= 1
    end_position = token_end_index + 1
    print(start_position, end_position)
else:
    print("The answer is not in this feature.")

16 17


In [296]:
print(
    tokenizer.decode(
        tokenized_example["input_ids"][0][start_position : end_position + 1]
    )
)
print(answers["text"][0])

jay z
Jay Z


In [297]:
pad_on_right = tokenizer.padding_side == "right"

In [298]:
def prepare_train_features(examples):
    # Tokenize our examples with truncation and padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")
    # The offset mappings will give us a map from token to character position in the original context. This will
    # help us compute the start_positions and end_positions.
    offset_mapping = tokenized_examples.pop("offset_mapping")

    # Let's label those examples!
    tokenized_examples["start_positions"] = []
    tokenized_examples["end_positions"] = []

    for i, offsets in enumerate(offset_mapping):
        # We will label impossible answers with the index of the CLS token.
        input_ids = tokenized_examples["input_ids"][i]
        cls_index = input_ids.index(tokenizer.cls_token_id)

        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        answers = examples["answers"][sample_index]
        # If no answers are given, set the cls_index as answer.
        if len(answers["answer_start"]) == 0:
            tokenized_examples["start_positions"].append(cls_index)
            tokenized_examples["end_positions"].append(cls_index)
        else:
            # Start/end character index of the answer in the text.
            start_char = answers["answer_start"][0]
            end_char = start_char + len(answers["text"][0])

            # Start token index of the current span in the text.
            token_start_index = 0
            while sequence_ids[token_start_index] != (1 if pad_on_right else 0):
                token_start_index += 1

            # End token index of the current span in the text.
            token_end_index = len(input_ids) - 1
            while sequence_ids[token_end_index] != (1 if pad_on_right else 0):
                token_end_index -= 1

            # Detect if the answer is out of the span (in which case this feature is labeled with the CLS index).
            if not (
                offsets[token_start_index][0] <= start_char
                and offsets[token_end_index][1] >= end_char
            ):
                tokenized_examples["start_positions"].append(cls_index)
                tokenized_examples["end_positions"].append(cls_index)
            else:
                # Otherwise move the token_start_index and token_end_index to the two ends of the answer.
                # Note: we could go after the last offset if the answer is the last word (edge case).
                while (
                    token_start_index < len(offsets)
                    and offsets[token_start_index][0] <= start_char
                ):
                    token_start_index += 1
                tokenized_examples["start_positions"].append(token_start_index - 1)
                while offsets[token_end_index][1] >= end_char:
                    token_end_index -= 1
                tokenized_examples["end_positions"].append(token_end_index + 1)

    return tokenized_examples

In [299]:
features = prepare_train_features(datasets["train"][:5])

In [300]:
tokenized_datasets = datasets.map(
    prepare_train_features, batched=True, remove_columns=datasets["train"].column_names
)

  0%|          | 0/115 [00:00<?, ?ba/s]

  0%|          | 0/12 [00:00<?, ?ba/s]

In [301]:
from transformers import TFAutoModelForQuestionAnswering

model = TFAutoModelForQuestionAnswering.from_pretrained(model_name)

All model checkpoint layers were used when initializing TFBertForQuestionAnswering.

Some layers of TFBertForQuestionAnswering were not initialized from the model checkpoint at cahya/bert-base-indonesian-522M and are newly initialized: ['qa_outputs']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [302]:
push_to_hub_model_id = "chatbot-trvlk-finetuned-squad"
learning_rate = 2e-5
num_train_epochs = 2
weight_decay = 0.01

In [303]:
from transformers import DefaultDataCollator

data_collator = DefaultDataCollator(return_tensors="tf")

In [304]:
train_set = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "start_positions", "end_positions"],
    shuffle=True,
    batch_size=batch_size,
    collate_fn=data_collator,
)
validation_set = tokenized_datasets["validation"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "start_positions", "end_positions"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [305]:
from transformers import create_optimizer

total_train_steps = (len(tokenized_datasets["train"]) // batch_size) * num_train_epochs

optimizer, schedule = create_optimizer(
    init_lr=learning_rate, num_warmup_steps=0, num_train_steps=total_train_steps
)

In [306]:
import tensorflow as tf

model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour, please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [307]:
from huggingface_hub import notebook_login

notebook_login()

Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your terminal in case you want to set this credential helper as the default

git config --global credential.helper store


In [308]:
!apt install git-lfs
!git config --global user.email "anandadaffa15@gmail.com"
!git config --global user.name "Andaf"

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git-lfs is already the newest version (2.3.4-1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 42 not upgraded.


In [309]:
from transformers.keras_callbacks import PushToHubCallback
from tensorflow.keras.callbacks import TensorBoard

push_to_hub_callback = PushToHubCallback(
    output_dir="./qa_model_save",
    tokenizer=tokenizer,
    hub_model_id=push_to_hub_model_id,
)

tensorboard_callback = TensorBoard(log_dir="./qa_model_save/logs")

callbacks = [tensorboard_callback, push_to_hub_callback]

model.fit(
    train_set,
    validation_data=validation_set,
    epochs=num_train_epochs,
    callbacks=callbacks,
)

/content/qa_model_save is already a clone of https://huggingface.co/Andaf/chatbot-trvlk-finetuned-squad. Make sure you pull the latest changes with `repo.git_pull()`.


Epoch 1/2
7222/7222 [==============================] - 6080s 840ms/step - loss: 2.1851 - val_loss: 6.1907
Epoch 2/2
7222/7222 [==============================] - 6058s 839ms/step - loss: 1.5335 - val_loss: 6.4566


Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file tf_model.h5:   0%|          | 3.34k/420M [00:00<?, ?B/s]

Upload file logs/validation/events.out.tfevents.1654148844.8b5c72b9c6a7.83.12.v2: 100%|##########| 350/350 [00…

Upload file logs/train/events.out.tfevents.1654142985.8b5c72b9c6a7.83.11.v2:   0%|          | 3.34k/2.55M [00:…

remote: Enforcing permissions...        
remote: Allowed refs: all        
To https://huggingface.co/Andaf/chatbot-trvlk-finetuned-squad
   2105b34..04c00a6  main -> main



In [310]:
model.save_pretrained("/content/drive/MyDrive/Translated/Model and Weight/model_trvlk")

In [323]:
 !zip -r qa_model_save.zip qa_model_save/

  adding: qa_model_save/ (stored 0%)
  adding: qa_model_save/config.json (deflated 46%)
  adding: qa_model_save/.gitattributes (deflated 86%)
  adding: qa_model_save/tokenizer.json (deflated 71%)
  adding: qa_model_save/tf_model.h5 (deflated 7%)
  adding: qa_model_save/vocab.txt (deflated 51%)
  adding: qa_model_save/.git/ (stored 0%)
  adding: qa_model_save/.git/description (deflated 14%)
  adding: qa_model_save/.git/objects/ (stored 0%)
  adding: qa_model_save/.git/objects/ac/ (stored 0%)
  adding: qa_model_save/.git/objects/ac/481c8eb05e4d2496fbe076a38a7b4835dd733d (stored 0%)
  adding: qa_model_save/.git/objects/e5/ (stored 0%)
  adding: qa_model_save/.git/objects/e5/43ec5dec53db3d45c58e0a9c432a4a00fcf018 (deflated 0%)
  adding: qa_model_save/.git/objects/pack/ (stored 0%)
  adding: qa_model_save/.git/objects/de/ (stored 0%)
  adding: qa_model_save/.git/objects/de/a8e1506604081af60e5e623dd2f694034c07d1 (stored 0%)
  adding: qa_model_save/.git/objects/d2/ (stored 0%)
  adding: qa_mo

In [ ]:
#dont use it
model.save('/content/drive/MyDrive/Translated/new_model')
# assign location
path='/content/drive/MyDrive/Translated/weight_new/Weights'
 
# saveweights
model.save_weights(path)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Translated/Model_trainindo/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Translated/Model_trainindo/assets


In [311]:
batch = next(iter(validation_set))
output = model.predict_on_batch(batch)
output.keys()

odict_keys(['loss', 'start_logits', 'end_logits'])

In [312]:
output.start_logits.shape, output.end_logits.shape

((16, 384), (16, 384))

In [313]:
import numpy as np

np.argmax(output.start_logits, -1), np.argmax(output.end_logits, -1)

(array([ 19,  33,  78,  87, 154,  20,   7,  11, 146, 181, 108,  38,  18,
         30,  29, 109]),
 array([ 19,  40,  82,  88, 157,  20,   7,  11, 161, 162, 111,  46,  22,
         30,  29, 110]))

In [314]:
n_best_size = 20

In [315]:
import numpy as np

start_logits = output.start_logits[0]
end_logits = output.end_logits[0]
# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        if (
            start_index <= end_index
        ):  # We need to refine that test to check the answer is inside the context
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": "",  # We need to find a way to get back the original substring corresponding to the answer in the context
                }
            )

In [316]:
def prepare_validation_features(examples):
    # Tokenize our examples with truncation and maybe padding, but keep the overflows using a stride. This results
    # in one example possible giving several features when a context is long, each of those features having a
    # context that overlaps a bit the context of the previous feature.
    tokenized_examples = tokenizer(
        examples["question" if pad_on_right else "context"],
        examples["context" if pad_on_right else "question"],
        truncation="only_second" if pad_on_right else "only_first",
        max_length=max_length,
        stride=doc_stride,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length",
    )

    # Since one example might give us several features if it has a long context, we need a map from a feature to
    # its corresponding example. This key gives us just that.
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    # We keep the example_id that gave us this feature and we will store the offset mappings.
    tokenized_examples["example_id"] = []

    for i in range(len(tokenized_examples["input_ids"])):
        # Grab the sequence corresponding to that example (to know what is the context and what is the question).
        sequence_ids = tokenized_examples.sequence_ids(i)
        context_index = 1 if pad_on_right else 0

        # One example can give several spans, this is the index of the example containing this span of text.
        sample_index = sample_mapping[i]
        tokenized_examples["example_id"].append(examples["id"][sample_index])

        # Set to None the offset_mapping that are not part of the context so it's easy to determine if a token
        # position is part of the context or not.
        tokenized_examples["offset_mapping"][i] = [
            (o if sequence_ids[k] == context_index else None)
            for k, o in enumerate(tokenized_examples["offset_mapping"][i])
        ]

    return tokenized_examples

In [317]:
validation_features = datasets["validation"].map(
    prepare_validation_features,
    batched=True,
    remove_columns=datasets["validation"].column_names,
)

  0%|          | 0/12 [00:00<?, ?ba/s]

In [318]:
validation_dataset = validation_features.to_tf_dataset(
    columns=["attention_mask", "input_ids"],
    shuffle=False,
    batch_size=batch_size,
    collate_fn=data_collator,
)

In [319]:
raw_predictions = model.predict(validation_dataset)

In [320]:
max_answer_length = 30

In [321]:
start_logits = output.start_logits[0]
end_logits = output.end_logits[0]
offset_mapping = validation_features[0]["offset_mapping"]
# The first feature comes from the first example. For the more general case, we will need to be match the example_id to
# an example index
context = datasets["validation"][0]["context"]

# Gather the indices the best start/end logits:
start_indexes = np.argsort(start_logits)[-1 : -n_best_size - 1 : -1].tolist()
end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
valid_answers = []
for start_index in start_indexes:
    for end_index in end_indexes:
        # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
        # to part of the input_ids that are not in the context.
        if (
            start_index >= len(offset_mapping)
            or end_index >= len(offset_mapping)
            or offset_mapping[start_index] is None
            or offset_mapping[end_index] is None
        ):
            continue
        # Don't consider answers with a length that is either < 0 or > max_answer_length.
        if end_index < start_index or end_index - start_index + 1 > max_answer_length:
            continue
        if (
            start_index <= end_index
        ):  # We need to refine that test to check the answer is inside the context
            start_char = offset_mapping[start_index][0]
            end_char = offset_mapping[end_index][1]
            valid_answers.append(
                {
                    "score": start_logits[start_index] + end_logits[end_index],
                    "text": context[start_char:end_char],
                }
            )

valid_answers = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
    :n_best_size
]
valid_answers

[{'score': 11.582796, 'text': 'Prancis'},
 {'score': 9.284663, 'text': 'Norman (Norman: Nourmands; Prancis'},
 {'score': 8.855457, 'text': 'Norman: Nourmands; Prancis'},
 {'score': 7.3117695, 'text': 'Prancis: Normandia'},
 {'score': 6.9868107, 'text': 'Norman'},
 {'score': 6.8033037, 'text': 'Bangsa Norman (Norman: Nourmands; Prancis'},
 {'score': 6.6759653, 'text': 'Norman (Norman'},
 {'score': 6.4182177, 'text': 'Prancis'},
 {'score': 6.2467585, 'text': 'Norman'},
 {'score': 5.6986036, 'text': 'Prancis: Normandia; Latin: Norman'},
 {'score': 5.4182086,
  'text': 'Prancis: Normandia; Latin: Normanni) adalah orang-orang yang pada abad ke-10 dan ke-11 memberi nama kepada Normandia'},
 {'score': 5.0136366, 'text': 'Norman (Norman: Nourmands; Prancis: Normandia'},
 {'score': 4.822694,
  'text': 'Normanni) adalah orang-orang yang pada abad ke-10 dan ke-11 memberi nama kepada Normandia, sebuah wilayah di Prancis'},
 {'score': 4.715297, 'text': 'Prancis.'},
 {'score': 4.58443, 'text': 'Norm

In [322]:
datasets["validation"][0]["answers"]

{'answer_start': [159, 159, 159, 159],
 'text': ['Perancis', 'Perancis', 'Perancis', 'Perancis']}

In [ ]:
model.save('/content/drive/MyDrive/Translated/latest_model')

# assign location
path='/content/drive/MyDrive/Translated/Weight_latest/Weights'
 
# saveweights
model.save_weights(path)

INFO:tensorflow:Assets written to: /content/drive/MyDrive/Translated/latest_model/assets


INFO:tensorflow:Assets written to: /content/drive/MyDrive/Translated/latest_model/assets


In [ ]:
tf.keras.models.load_model('')

In [ ]:
# python
from transformers import TFAutoModel
# bert = TFAutoModel.from_pretrained("bert-base-uncased")
bert = TFAutoModel.from_pretrained("/content/drive/MyDrive/Translated/Model and Weight/latest_model")

OSError: ignored

In [ ]:
# After this Code and below dont know if it necessary or not
import collections

examples = datasets["validation"]
features = validation_features

example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
features_per_example = collections.defaultdict(list)
for i, feature in enumerate(features):
    features_per_example[example_id_to_index[feature["example_id"]]].append(i)

In [ ]:
from tqdm.auto import tqdm


def postprocess_qa_predictions(
    examples,
    features,
    all_start_logits,
    all_end_logits,
    n_best_size=20,
    max_answer_length=30,
):
    # Build a map example to its corresponding features.
    example_id_to_index = {k: i for i, k in enumerate(examples["id"])}
    features_per_example = collections.defaultdict(list)
    for i, feature in enumerate(features):
        features_per_example[example_id_to_index[feature["example_id"]]].append(i)

    # The dictionaries we have to fill.
    predictions = collections.OrderedDict()

    # Logging.
    print(
        f"Post-processing {len(examples)} example predictions split into {len(features)} features."
    )

    # Let's loop over all the examples!
    for example_index, example in enumerate(tqdm(examples)):
        # Those are the indices of the features associated to the current example.
        feature_indices = features_per_example[example_index]

        min_null_score = None  # Only used if squad_v2 is True.
        valid_answers = []

        context = example["context"]
        # Looping through all the features associated to the current example.
        for feature_index in feature_indices:
            # We grab the predictions of the model for this feature.
            start_logits = all_start_logits[feature_index]
            end_logits = all_end_logits[feature_index]
            # This is what will allow us to map some the positions in our logits to span of texts in the original
            # context.
            offset_mapping = features[feature_index]["offset_mapping"]

            # Update minimum null prediction.
            cls_index = features[feature_index]["input_ids"].index(
                tokenizer.cls_token_id
            )
            feature_null_score = start_logits[cls_index] + end_logits[cls_index]
            if min_null_score is None or min_null_score < feature_null_score:
                min_null_score = feature_null_score

            # Go through all possibilities for the `n_best_size` greater start and end logits.
            start_indexes = np.argsort(start_logits)[
                -1 : -n_best_size - 1 : -1
            ].tolist()
            end_indexes = np.argsort(end_logits)[-1 : -n_best_size - 1 : -1].tolist()
            for start_index in start_indexes:
                for end_index in end_indexes:
                    # Don't consider out-of-scope answers, either because the indices are out of bounds or correspond
                    # to part of the input_ids that are not in the context.
                    if (
                        start_index >= len(offset_mapping)
                        or end_index >= len(offset_mapping)
                        or offset_mapping[start_index] is None
                        or offset_mapping[end_index] is None
                    ):
                        continue
                    # Don't consider answers with a length that is either < 0 or > max_answer_length.
                    if (
                        end_index < start_index
                        or end_index - start_index + 1 > max_answer_length
                    ):
                        continue

                    start_char = offset_mapping[start_index][0]
                    end_char = offset_mapping[end_index][1]
                    valid_answers.append(
                        {
                            "score": start_logits[start_index] + end_logits[end_index],
                            "text": context[start_char:end_char],
                        }
                    )

        if len(valid_answers) > 0:
            best_answer = sorted(valid_answers, key=lambda x: x["score"], reverse=True)[
                0
            ]
        else:
            # In the very rare edge case we have not a single non-null prediction, we create a fake prediction to avoid
            # failure.
            best_answer = {"text": "", "score": 0.0}

        # Let's pick our final answer: the best one or the null answer (only for squad_v2)
        if not squad_v2:
            predictions[example["id"]] = best_answer["text"]
        else:
            answer = (
                best_answer["text"] if best_answer["score"] > min_null_score else ""
            )
            predictions[example["id"]] = answer

    return predictions

In [ ]:
final_predictions = postprocess_qa_predictions(
    datasets["validation"],
    validation_features,
    raw_predictions["start_logits"],
    raw_predictions["end_logits"],
)